In [2]:
from src.utils.trie_index import find_next_word_continuations, get_distinct_tokens_from_trie

trie_index_path = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"
results = find_next_word_continuations("Aram", trie_index_path)
print("Results for 'Aram':")
print(results)

Results for 'Aram':
[' ,', ' ]', ' khachaturian', ' khatchaturian', ' manukian', '_', '_khachaturian', '_khatchaturian', '_manukian', 'aic', 'aic_language', 'aic_script', 'ais', 'aized', 'ay', 'azd', 'bagh', 'barri', 'e', 'ean', 'eans', 'eans_in_israel', 'id', 'is', 'is_knight', 'is_rami', 'yan']


In [3]:
import transformers
from guidance import models
import torch

MODEL_NAME = "/data/hdd1/users/akouk/ARM/ARM/assets/cache/models--Qwen--Qwen2.5-32B-Instruct"
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = models.Transformers(
    MODEL_NAME,
    echo=True,
    device_map='auto',         
    torch_dtype=torch.float16,
    repetition_penalty=1.5
    
)

2025-05-08 15:29:26,741 - INFO - Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2025-05-08 15:29:26,743 - INFO - Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-05-08 15:29:26,744 - INFO - NumExpr defaulting to 16 threads.


[2025-05-08 15:29:28,531] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-05-08 15:29:28,596 - INFO - /home/akouk/miniconda3/envs/tolis/bin/x86_64-conda-linux-gnu-cc -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/akouk/miniconda3/envs/tolis/include -fPIC -O2 -isystem /home/akouk/miniconda3/envs/tolis/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/akouk/miniconda3/envs/tolis/include -I/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/include -L/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/lib -L/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/lib/stubs -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/akouk/miniconda3/envs/tolis/include -I/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/include -L/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/lib -L/home/akouk/miniconda3/envs/tolis/targets/x86_64-linux/lib/stubs -fPIC -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [4]:
def configure_list_size(string_list, n):
    if not string_list:
        return []
    if len(string_list) <= n:
        return string_list
    max_s_len = 0
    for s_item in string_list:
        if len(s_item) > max_s_len:
            max_s_len = len(s_item)
    optimal_trunc_len = 0
    low = 0
    high = max_s_len

    while low <= high:
        mid_len = low + (high - low) // 2
        
        truncated_set = {s_val[:mid_len] for s_val in string_list}
        
        if len(truncated_set) <= n:
            optimal_trunc_len = mid_len
            low = mid_len + 1 
        else:
            high = mid_len - 1
            
    final_truncated_set = {s_val[:optimal_trunc_len] for s_val in string_list}
    return list(final_truncated_set)

In [5]:
from src.utils.trie_index import find_next_word_continuations, get_distinct_tokens_from_trie

from src.retrieval.base import RetrievalResult
from src.retrieval.bm25 import PyseriniBM25Retriever
from src.retrieval.dense import FaissDenseRetriever
from typing import List
import guidance
from guidance import models, gen, select, capture
import re
from typing import List, Callable
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

"""trie_index_path = "assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"
prefix = "iver"
continuations = find_next_word_continuations(prefix, trie_index_path)
print(f"\n--- Next word continuations: {continuations}")
all_tokens = get_distinct_tokens_from_trie(trie_index_path)
print(f"\n--- Total distinct tokens in the Trie: {len(all_tokens)} ---")
bm25 = PyseriniBM25Retriever()
dense_index = FaissDenseRetriever()
retrieved_results_bm25 : List[List[RetrievalResult]] = []
retrieved_results_bm25 += bm25.retrieve(
    nlqs=["Formula E"],
    output_folder="assets/feverous/pyserini_indexes/bm25_row_index",
    k=5
)
retrieved_results_dense : List[List[RetrievalResult]] = []
retrieved_results_dense += dense_index.retrieve(
    nlqs=["Formula E"],
    output_folder="assets/feverous/faiss_indexes/dense_row_UAE-Large-V1",
    k=5
)

print("\n--- Retrieved Results with BM25 ---")
for i, result in enumerate(retrieved_results_bm25):
    for res in result:
        print(res.object)
print("\n\n--- Retrieved Results with Dense ---") 
for i, result in enumerate(retrieved_results_dense):
    for res in result:
        print(f"From {res.metadata.{res.object}")
"""



trie_index_path = "/data/hdd1/users/akouk/ARM/ARM/assets/feverous/trie_indexes/ngrams_table_level_1_3.marisa"
dense_index = FaissDenseRetriever()

max_keyword_length = 20
max_rephreases = 5
@guidance(stateless=False) # Needs to be stateful to use Python logic based on captures
def dynamic_retrieval_guidance(lm, user_query: str):

    # 1. Decompose query into keywords
    lm += f"""You are given a user question, your task is to decompose the user question into contiguous, non-overlapping substrings that can
cover different information mentioned in the user question. For each substring, generate n-grams that are the most relevant to
the substring. Prefare lengthy n-grams over short ones. Based on the generated relevant n-grams, generate a list of relevant objects, including their names, content, and
connections between these objects. From these candidate objects, you should identify the minimum number of objects that can
be used to answer the user question based on the relevance between the object name, object content and user question as well as
the relevance of the object connections. You should end your response with <>.

User question: What is the birth date of the director of the movie 'Inception' which was released in 2010?
The relevant keywords are birth date | director | movie 'Inception' | released in 2010
The relevant n-grams are birth date (born, date of birth, DOB) | director (directed by, filmmaker, film director) | movie 'Inception' (Inception, film Inception, movie named Inception) | released in 2010 (2010, release year 2010, came out 2010)

Here are the objects that can be relevant:
(...4 objects omitted...)
- Inception was directed by Christopher Nolan.
- Lions live in the jungle.


Here are the objects that are enough to answer the user query:
(...2 objects omitted...)
- Inception was directed by Christopher Nolan.


User question: {user_query}/no_think
The relevant keywords are: """
    lm += gen(name='keywords_str', stop='\n')

    # Parse keywords (Python logic outside LLM generation flow but within the guidance function)
    keywords_str = lm['keywords_str'].strip()
    keywords = [k.strip() for k in keywords_str.split('|')]
    print(f"--- Identified Keywords: {keywords} ---")

    # 2. Rephrase keywords using dynamic constraints
    lm += "\nThe relevant n-grams are"
    rephrased_dict = {}
    for keyword_idx, keyword in enumerate(keywords): # Use enumerate for unique variable names
        if keyword_idx == 0:
            lm += f" {keyword} (" # Start the rephrasing block for this keyword
        else:
            lm += f" | {keyword} ("
        next_keyword = False
        i = 0
        while not next_keyword and i < max_rephreases:
            i += 1
            first_token_var = f"rephrase_{keyword_idx}_first_token"
            lm += gen(name=first_token_var,max_tokens=1)
            current_rephrase = lm[first_token_var]
            if len(current_rephrase) < 2:
                second_token_var = f"rephrase_{keyword_idx}_second_token"
                lm += gen(name=second_token_var, max_tokens=1)
                current_rephrase += lm[second_token_var]
            step = 0    
            while True and step < max_keyword_length:
                continuations = find_next_word_continuations(current_rephrase, trie_index_path)
                #continuations = [token.lower() for token in continuations]
                #continuations = [token for token in continuations if all(char in string.ascii_lowercase for char in token)]
                #continuations = [
                #    s for s in continuations
                #    if len(ids := tokenizer.encode(s, add_special_tokens=False)) == 1 and \
                #    tokenizer.decode(ids[0]) == s
                #]
                valid_next_strings = continuations + [')', ', ', ' '] 
                valid_next_strings = list(set(valid_next_strings))
                valid_next_strings = [s for s in valid_next_strings if s != '|']
                valid_next_strings = configure_list_size(valid_next_strings, 2000)
                current_token_var = f"rephrase_{keyword_idx}_token_{step}"                
                lm += select(options=valid_next_strings, name=current_token_var)
                selected_token_str = lm[current_token_var]
                step += 1
                current_rephrase += selected_token_str
                if selected_token_str == ')':
                    next_keyword = True
                    break
                if selected_token_str == ', ':
                    break


    lm = lm[:-2] 

    # Store the rephrased keywords in the lm state for potential later use
    lm = lm.set("rephrased_dict", rephrased_dict)

    # 3. Retrieval (Python logic)
    retrieved_results_dense: List[List[RetrievalResult]] = []
    all_retrieved_objects: List[str] = []

    # Prepare nlqs list for retrieval call
    nlqs_for_retrieval = [v for v in rephrased_dict.values() if v] # Use non-empty rephrased terms
    if nlqs_for_retrieval:
        retrieved_results_dense = dense_index.retrieve(
            nlqs=nlqs_for_retrieval, # Pass the list of rephrased keywords/n-grams
            output_folder="assets/feverous/faiss_indexes/dense_row_UAE-Large-V1", # Example path
            k=5
        )
        # Flatten the results and get object strings
        for result_list in retrieved_results_dense:
            for res in result_list:
                all_retrieved_objects.append(res.object)
        all_retrieved_objects = list(set(all_retrieved_objects)) # Make unique
    else:
        print("--- No valid rephrased keywords for retrieval ---")


    # 4. List all retrieved objects (Forced Generation)
    lm += "\n\nHere are the objects that can be relevant:\n"
    if all_retrieved_objects:
        for obj_text in all_retrieved_objects:
            lm += f"- {obj_text}\n" # Force the LLM to output each object string
    else:
        lm += "- (No objects retrieved)\n"

    # Store all retrieved objects text in the lm state
    lm = lm.set("all_retrieved_objects", all_retrieved_objects)

    # 5. Filter relevant objects
    lm += "\nHere are the objects that are enough to answer the user query:\n"
    # Let the LLM generate the filtered list based on the original query and the full list provided above
    lm += gen(name='filtered_objects', stop='<>', max_tokens=500) # Generate until the final stop token

    return lm
import transformers
user_query = "Aramais Yepiskoposan played for FC Ararat Yerevan, an Armenian football club based in Yerevan during 1986 to 1991."

executed_program = model + dynamic_retrieval_guidance(user_query)

# Print the final generated output
print("\n--- Final LLM Output ---")
print(str(executed_program))

2025-05-08 15:29:49,455 - INFO - Loading faiss with AVX512 support.
2025-05-08 15:29:49,471 - INFO - Successfully loaded faiss with AVX512 support.
2025-05-08 15:29:49,522 - INFO - Load pretrained SentenceTransformer: WhereIsAI/UAE-Large-V1


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

--- Identified Keywords: ['Aramais Yepiskoposan', 'played for', 'FC Ararat Yerevan', 'Armenian football club', 'based in Yerevan', '1986 to 1991'] ---


TypeError: unhashable type: 'slice'